https://arxiv.org/pdf/2108.06973.pdf

https://towardsdatascience.com/item-based-collaborative-filtering-in-python-91f747200fab




In [ ]:
"""
Colab execution only
"""
import os
from pathlib import Path
!pip install wget
!git clone "https://github.com/antoniosh97/Recommender-System-2023.git"

path = Path('Recommender-System-2023/Implementation/3_LabReplication')
print(f"Current path: \n{path}\nContent inside the folder:\n{os.listdir(path)}")
os.chdir(path)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
fatal: destination path 'Recommender-System-2023' already exists and is not an empty directory.
Current path: 
Recommender-System-2023/Implementation/3_LabReplication
Content inside the folder:
['DatasetPoint.py', 'RandomEval.py', 'FactorizationMachines.py', '00_FM_and_Random_FullSampling.ipynb', 'Unified_Evaristo']


In [1]:
#====================== Import de librerias =====================#

import time
from pathlib import Path
import json
import gzip
from urllib.request import urlopen
import datetime
import plotly.express as px
import plotly.graph_objects as go
import wget
import logging
from tqdm import tqdm
import random
import time


import torch
import pandas as pd
import numpy as np
import csv
import os
import scipy.sparse as sp
from typing import Tuple, Dict, Any, List
from tqdm import tqdm, trange
from IPython import embed
from torch.utils.data import DataLoader, Dataset
from torch.utils.tensorboard import SummaryWriter
import tensorboard
import webbrowser

sampling_method = "StratifiedSamplingByItem"
# old_path = os.getcwd()
# os.chdir("..")
# execution_path = os.getcwd()

if not os.getcwd().split(os.sep)[-1] == "Implementation":
    os.chdir("..")
execution_path = os.getcwd()


In [ ]:
execution_path

'/content/Recommender-System-2023/Implementation'

In [ ]:
%load_ext tensorboard

logs_base_dir = "runs_"+sampling_method
os.environ["run_tensorboard"] = logs_base_dir

os.makedirs(f'{execution_path}/{"4_Modelling"}/{logs_base_dir}', exist_ok=True)
tb_fm = SummaryWriter(log_dir=f'{execution_path}/{"4_Modelling"}/{logs_base_dir}/{logs_base_dir}_FM/')
tb_rnd = SummaryWriter(log_dir=f'{execution_path}/{"4_Modelling"}/{logs_base_dir}/{logs_base_dir}_RANDOM/')

def save_data_configuration(text):
    save_data_dir = "data_config_" + sampling_method +".txt"
    path = f'{execution_path}/{"4_Modelling"}/{save_data_dir}'
    with open(path, "a") as data_file:
        data_file.write(text+"\n")

    return text

In [ ]:
# Let's define some hyper-parameters
hparams = {
    'batch_size':64,
    'num_epochs':12,
    'hidden_size': 32,
    'learning_rate':1e-4,
}

# we select to work on GPU if it is available in the machine, otherwise
# will run on CPU
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [ ]:
#============ Definicion de valores de configuracion ============#

min_reviews, min_usuarios = [6,6]
col_names = {"col_id_reviewer": "reviewerID",
             "col_id_product": "asin",
             "col_unix_time": "unixReviewTime",
             "col_rating": "overall",
             "col_timestamp": "timestamp",
             "col_year": "year"}

csv_filename = execution_path/Path("3_DataPreparation/interactions_minR{}_minU{}.csv".format(min_reviews,min_usuarios))

In [ ]:
df = pd.read_csv(csv_filename)
df.head()

,asin,reviewerID,overall,unixReviewTime,timestamp,year
0,0,9132,5.0,1477785600,2016-10-30 02:00:00,1970
1,0,10612,5.0,1467244800,2016-06-30 02:00:00,1970
2,0,257,1.0,1454716800,2016-02-06 01:00:00,1970
3,0,4425,5.0,1434844800,2015-06-21 02:00:00,1970
4,0,2523,4.0,1420329600,2015-01-04 01:00:00,1970


In [ ]:
save_data_configuration(str(df.nunique()))
df.nunique()

asin               6178
reviewerID        14138
overall               5
unixReviewTime     3622
timestamp          3622
year                  1
dtype: int64

# Splitting dataset (TLOO strategy)

In [ ]:
# def split_train_test(data: np.ndarray,
#                      n_users: int) -> Tuple[np.ndarray, np.ndarray]:
#     # Split and remove timestamp
#     train_x, test_x = [], []
#     for u in trange(n_users, desc='spliting train/test and removing timestamp...'):
#         user_data = data[data[:, 0] == u]
#         sorted_data = user_data[user_data[:, -1].argsort()]
#         if len(sorted_data) == 1:
#             train_x.append(sorted_data[0][:-1])
#         else:
#             train_x.append(sorted_data[:-1][:, :-1])
#             test_x.append(sorted_data[-1][:-1])
#     return np.vstack(train_x), np.stack(test_x)

def split_train_test(data: np.ndarray,
                     n_users: int) -> Tuple[np.ndarray, np.ndarray]:
    # Split and remove timestamp
    train_x, test_x = [], []
    for u in trange(n_users, desc='spliting train/test and removing timestamp...'):
        user_data = data[data[:, 0] == u]
        sorted_data = user_data[user_data[:, -1].argsort()] # ordenamos temporalmente
        if len(sorted_data) == 1:
            train_x.append(sorted_data[0][:1])
        else:
            train_x.append(sorted_data[:-1][:,:-1])
            test_x.append(sorted_data[-1][:-1])
    return np.vstack(train_x), np.stack(test_x)

In [ ]:

col_names = {"col_id_reviewer": "reviewerID",
             "col_id_product": "asin",
             "col_rating": "overall",
             "col_unix_time": "unixReviewTime",
             "col_timestamp": "timestamp",
             "col_year": "year"}

In [ ]:
# data = df[[*col_names.values()][:3]].astype('int32').to_numpy()
# data
data = df[[*col_names.values()][:4]].astype('int32').to_numpy()
data

array([[      9132,          0,          5, 1477785600],
       [     10612,          0,          5, 1467244800],
       [       257,          0,          1, 1454716800],
       ...,
       [      9051,       6177,          5, 1530144000],
       [      3412,       6177,          5, 1527465600],
       [      9805,       6177,          5, 1527206400]], dtype=int32)

In [ ]:
# add_dims=0
# for i in range(data.shape[1] - 1):  # do not affect to timestamp
#     # MAKE IT START BY 0
#     data[:, i] -= np.min(data[:, i])
#     # RE-INDEX
#     data[:, i] += add_dims
#     add_dims = np.max(data[:, i]) + 1
# dims = np.max(data, axis=0) + 1
# print("Dim of users: {}\nDim of items: {}\nDims of unixtime: {}".format(dims[0], dims[1], dims[2]))
# data

add_dims=0
for i in range(data.shape[1] - 2):  # do not affect to timestamp and rating
    # MAKE IT START BY 0
    data[:, i] -= np.min(data[:, i])
    # RE-INDEX
    data[:, i] += add_dims
    add_dims = np.max(data[:, i]) + 1
dims = np.max(data, axis=0) + 1
print("Dim of users: {}\nDim of items: {}\nDims of unixtime: {}".format(dims[0], dims[1], dims[2]))
data

Dim of users: 14138
Dim of items: 20316
Dims of unixtime: 6


array([[      9132,      14138,          5, 1477785600],
       [     10612,      14138,          5, 1467244800],
       [       257,      14138,          1, 1454716800],
       ...,
       [      9051,      20315,          5, 1530144000],
       [      3412,      20315,          5, 1527465600],
       [      9805,      20315,          5, 1527206400]], dtype=int32)

In [ ]:
train_x, test_x = split_train_test(data, dims[0])
train_x

spliting train/test and removing timestamp...: 100%|██████████| 14138/14138 [00:09<00:00, 1452.02it/s]


array([[    0, 19248,     5],
       [    0, 19249,     5],
       [    0, 14823,     4],
       ...,
       [14137, 14159,     5],
       [14137, 18245,     5],
       [14137, 18904,     5]], dtype=int32)

# Negative sampling

In [ ]:
test_x

array([[    0, 17249,     5],
       [    1, 18015,     5],
       [    2, 14196,     4],
       ...,
       [14135, 19938,     5],
       [14136, 20214,     2],
       [14137, 15542,     3]], dtype=int32)

In [ ]:
# train_x = train_x[:, :2] # si quieres quitarte el rating
dims = dims[:2]
print("New dims:",dims)
print("New train_x:\n",train_x)

New dims: [14138 20316]
New train_x:
 [[    0 19248     5]
 [    0 19249     5]
 [    0 14823     4]
 ...
 [14137 14159     5]
 [14137 18245     5]
 [14137 18904     5]]


In [ ]:
def build_adj_mx(n_feat:int, data:np.ndarray) -> sp.dok_matrix :
    train_mat = sp.dok_matrix((n_feat, n_feat), dtype=np.float32)
    for x in tqdm(data, desc=f"BUILDING ADJACENCY MATRIX..."):
        train_mat[x[0], x[1]] = 1.0
        train_mat[x[1], x[0]] = 1.0
        # IDEA: We treat features that are not user or item differently because we do not consider
        #  interactions between contexts
        if data.shape[1] > 2:
            for idx in range(len(x[2:])):
                train_mat[x[0], x[2 + idx]] = 1.0
                train_mat[x[1], x[2 + idx]] = 1.0
                train_mat[x[2 + idx], x[0]] = 1.0
                train_mat[x[2 + idx], x[1]] = 1.0
    return train_mat

In [ ]:
def ng_sample(data: np.ndarray, dims: list, num_ng:int=4) -> Tuple[np.ndarray, sp.dok_matrix]:
    rating_mat = build_adj_mx(dims[-1], data)
    interactions = []
    min_item, max_item = dims[0], dims[1]
    for num, x in tqdm(enumerate(data), desc='perform negative sampling...'):
        interactions.append(np.append(x, 1))
        for t in range(num_ng):
            j = np.random.randint(min_item, max_item) #if not pop else random.sample(items_to_sample, 1)[0]
            # IDEA: Loop to exclude true interactions (set to 1 in adj_train) user - item
            while (x[0], j) in rating_mat or j == int(x[1]):
                j = np.random.randint(min_item, max_item) #if not pop else random.sample(items_to_sample, 1)[0]
            interactions.append(np.concatenate([[x[0], j], x[2:], [0]]))
    return np.vstack(interactions), rating_mat

In [ ]:
def get_popularity_recommendations(train_x, col_names, dims):
  """
  https://github.com/LaxmiChaudhary/Amzon-Product-Recommendation/blob/master/Recommendation%20System.ipynb
  """
  train_data_sort = pd.DataFrame(train_x, columns=list(col_names.values())[:3]).sort_values(col_names['col_rating'], ascending=False)
  train_data_sort['rank'] = train_data_sort[col_names['col_id_product']].rank(ascending=0, method='first')
  pop_rec = train_data_sort[col_names['col_id_product']].drop_duplicates().to_numpy()
  print(pop_rec)
  pop_rec = pop_rec - dims[0]
  print(pop_rec)
  return pop_rec

popularity_recommendations = get_popularity_recommendations(train_x, col_names, dims)
popularity_recommendations
# #Sort the products on recommendation score 
# train_data_sort = train_data_grouped.sort_values(['score', 'productId'], ascending = [0,1]) 
      
# #Generate a recommendation rank based upon score 
# train_data_sort['rank'] = train_data_sort['score'].rank(ascending=0, method='first') 
          
# #Get the top 5 recommendations 
# popularity_recommendations = train_data_sort.head(5) 
# popularity_recommendations 


[19248 17630 14914 ... 20174 18289 16641]
[5110 3492  776 ... 6036 4151 2503]


array([5110, 3492,  776, ..., 6036, 4151, 2503], dtype=int32)

In [ ]:

train_x, rating_mat = ng_sample(train_x, dims)
print("Dimensions matrix:\n",dims)
print("\nRating matrix:")
rating_mat

BUILDING ADJACENCY MATRIX...: 100%|██████████| 123226/123226 [00:39<00:00, 3133.91it/s]
perform negative sampling...: 123226it [00:08, 15037.00it/s]


Dimensions matrix:
 [14138 20316]

Rating matrix:


<20316x20316 sparse matrix of type '<class 'numpy.float32'>'
	with 347600 stored elements in Dictionary Of Keys format>

In [ ]:
dims[-1]-dims[0]

6178

In [ ]:
# Exercise 2

## Evaristo
#### number of ones
print(np.count_nonzero(rating_mat.toarray())/(dims[-1]*dims[-1]))
### number of zeros
print(1 - np.count_nonzero(rating_mat.toarray())/(dims[-1]*dims[-1]))

# ## Brenda
# #### Who sparse is the matrix??
# print(1 - rating_mat.shape[0] / rating_mat.count_nonzero())

0.0008421769667914019
0.9991578230332085


In [ ]:
train_x[:10]

array([[    0, 19248,     5,     1],
       [    0, 19642,     5,     0],
       [    0, 16204,     5,     0],
       [    0, 14240,     5,     0],
       [    0, 16743,     5,     0],
       [    0, 19249,     5,     1],
       [    0, 14990,     5,     0],
       [    0, 14515,     5,     0],
       [    0, 17918,     5,     0],
       [    0, 15626,     5,     0]])

# Creating dataset class

In [ ]:
class PointData(Dataset):
    def __init__(self,
                 data: np.ndarray,
                 dims: list) -> None:
        """
        Dataset formatter adapted point-wise algorithms
        Parameters
        """
        super(PointData, self).__init__()
        self.interactions = data
        self.dims = dims

    def __len__(self) -> int:
        return len(self.interactions)
        
    def __getitem__(self, 
                    index: torch.Tensor) -> Tuple[torch.Tensor, torch.Tensor]:
        """
        Return the pairs user-item and the target.
        """
        return self.interactions[index][:-1], self.interactions[index][-1]

train_dataset = PointData(train_x, dims)

In [ ]:
train_dataset[0]

(array([    0, 19248,     5]), 1)

# Preparing the test set for inference

In [ ]:
test_x

array([[    0, 17249,     5],
       [    1, 18015,     5],
       [    2, 14196,     4],
       ...,
       [14135, 19938,     5],
       [14136, 20214,     2],
       [14137, 15542,     3]], dtype=int32)

In [ ]:
import math
print(rating_mat.shape)
bits = math.ceil(math.log(rating_mat.shape[0],2))
print("rating_mat contains log2(rating_mat.shape[0]) = {} bits".format(bits))

(20316, 20316)
rating_mat contains log2(rating_mat.shape[0]) = 15 bits


In [ ]:
def create_test_no_interactions_stratified_items(train_x: np.ndarray, test_x: np.ndarray, dims, num_samples):
    save_data_configuration("\n"+"#"*4+"  zero_positions: Stratified Sampling By Items  "+"#"*4)

    seed = 2   # get same results
    random.seed(a = seed)

    items_test = np.unique(test_x[:, 1])
    total_users = range(0,dims[0])
    zero_positions = np.zeros((num_samples * len(items_test), 2)).astype(int)
    start_index = 0

    for item in tqdm(items_test):
        strata_users_train = np.unique(train_x[train_x[:,1] == item][:,0])
        strata_users_test = np.unique(test_x[test_x[:,1] == item][:,0])
        random_selection_users = random.choices(list(set(total_users) - set(strata_users_train) - set(strata_users_test)), k=num_samples) # random.sample

        zero_positions[start_index:start_index + num_samples, 0] = random_selection_users
        zero_positions[start_index:start_index + num_samples, 1] = item
        start_index += num_samples

    indexes = np.argsort(zero_positions[:,0])
    # [00:14<00:00, 325.18it/s]
    zero_positions[:,0] = zero_positions[indexes,0]
    zero_positions[:,1] = zero_positions[indexes,1]
    return zero_positions
    # [00:14<00:00, 310.87it/s]
    # return np.vstack([zero_positions[indexes,0],zero_positions[indexes,1]]).T

In [ ]:
# def create_test_no_interactions_stratified_itemss(train_x: np.ndarray, test_x: np.ndarray, dims_usuarios_productos: Tuple[int, int],  num_samples: int) -> np.ndarray:
#     """
#     Esta funcion se encarga de crear de manera eficiente un dataset que contenga las interacciones usuario-producto en test que no se hayan producido.
    
#     Argumentos:
#         train_x (np.ndarray): matriz de entrenamiento con las interacciones usuario-producto previas
#         test_x (np.ndarray): matriz de prueba con las interacciones usuario-producto previas
#         dims_usuarios_productos (Tuple[int, int]): rango de productos y usuarios disponibles
    
#     Retorno:
#         np.ndarray: una matriz con todas las interacciones usuario-producto en test que no se hayan producido
#     """
#     from tqdm import tqdm
#     import random
    
#     # Identificamos los usuarios presentes en la prueba
#     usuarios_test = np.unique(test_x[:, 0])
#     # Identificamos el rango de productos disponibles
#     total_productos = range(dims_usuarios_productos[0]-1, dims_usuarios_productos[1])
    
#     # Recorremos cada usuario presente en la prueba
#     for usuario in tqdm(usuarios_test):
#         productos_train = np.unique(train_x[train_x[:, 0] == usuario][:, 1])
#         productos_a_machear = random.choices(list(set(total_productos) - set(productos_train)), k=num_samples)
#         lista_por_usuario = np.vstack([(np.ones(len(productos_a_machear))*usuario).astype(int), productos_a_machear]).T
        
#         # Si es el primer usuario, inicializamos una matriz con sus interacciones
#         if usuario == 0:
#             zero_positions = lista_por_usuario
#         # Si no es el primer usuario, concatenamos sus interacciones a la matriz existente
#         else:
#             zero_positions = np.concatenate((zero_positions, lista_por_usuario), axis=0)
            
#     return zero_positions

In [ ]:
# def zero_positions_mode(mode, rating_mat, train_x, test_x, dims):
   
#     if mode == 0:
#         save_data_configuration("\n"+"#"*4+"  zero_positions: all data  "+"#"*4)
#         return np.asarray(np.where(rating_mat.A==0)).T
#     elif mode == 1:
#         zero_true_matrix = np.where(rating_mat.A==0)
#         save_data_configuration("\n"+"#"*4+"  zero_positions: all data separated by rows  "+"#"*4)
#         return np.asarray([zero_true_matrix[0],zero_true_matrix[1]]).T
#     else:
#         save_data_configuration("\n"+"#"*4+"  zero_positions: random sampling  "+"#"*4)
#         zp = create_test_no_interactions(train_x, test_x, dims,  num_samples=199)
#         return zp

In [ ]:
def zero_pos(rating_mat):
    zero_true_matrix = np.where(rating_mat.A==0)
    save_data_configuration("\n"+"#"*4+"  zero_positions: all data separated by rows  "+"#"*4)
    return np.asarray([zero_true_matrix[0],zero_true_matrix[1]]).T
zero_positions = zero_pos(rating_mat)

In [30]:

num_samples = 400 # number of users in strata
zero_positions = create_test_no_interactions_stratified_items(train_x, test_x, dims,num_samples)
print(save_data_configuration(str(zero_positions.shape)+f"\t-----> num_samples = {num_samples}\n"))
zero_positions

100%|██████████| 4654/4654 [00:22<00:00, 210.18it/s]


(1861600, 2)	-----> num_samples = 400



array([[    0, 14439],
       [    0, 19776],
       [    0, 17584],
       ...,
       [14137, 18730],
       [14137, 19181],
       [14137, 14841]])

In [31]:
"""
Esta parte del código se usaba porque el zero positions venia de la rating matrix la cual tenia informacion no útil como (user,user) o (item,item), por eso nos quedábamos con items mayores a dims[0]
Ya que si recordamos, la matrix tenia size: (users + items)
rango items iniciales = 0 ... 6177
rango items actuales  = 14138 ... 20135
"""

# items2compute = []
# for user in trange(dims[0]):
#     aux = zero_positions[zero_positions[:, 0] == user][:, 1]
#     items2compute.append(aux[aux >= dims[0]])
# items2compute[0]


items2compute = []
for user in trange(dims[0]):
    # aux = np.array(sorted(zero_positions[start:start+num_samples,1]))
    # [00:36<00:00, 386.85it/s]	-----> num_samples = 199
    # [01:39<00:00, 141.49it/s]	-----> num_samples = 456
    aux = np.array(sorted(zero_positions[zero_positions[:, 0] == user][:, 1]))
    items2compute.append(aux.copy())
items2compute[0]

100%|██████████| 14138/14138 [01:58<00:00, 119.51it/s]


array([14149, 14156, 14158, 14177, 14208, 14221, 14271, 14280, 14346,
       14402, 14439, 14463, 14471, 14489, 14502, 14550, 14624, 14647,
       14695, 14722, 14756, 14867, 14886, 14905, 14979, 14996, 15012,
       15023, 15077, 15142, 15146, 15287, 15317, 15452, 15466, 15505,
       15544, 15549, 15566, 15577, 15691, 15849, 15887, 15942, 15958,
       15978, 16007, 16065, 16129, 16300, 16338, 16414, 16439, 16451,
       16491, 16685, 16723, 16725, 16773, 16811, 16832, 16832, 16895,
       16914, 16951, 16953, 16956, 16966, 17034, 17045, 17050, 17061,
       17138, 17178, 17243, 17346, 17411, 17470, 17496, 17545, 17561,
       17581, 17584, 17631, 17641, 17644, 17659, 17795, 17912, 17934,
       17972, 17994, 18107, 18146, 18149, 18210, 18211, 18241, 18247,
       18403, 18428, 18540, 18604, 18621, 18734, 18760, 18802, 18806,
       18844, 18852, 18869, 18876, 18895, 18913, 18965, 18979, 19082,
       19102, 19132, 19175, 19228, 19300, 19312, 19316, 19317, 19321,
       19325, 19384,

In [32]:
test_x[:,:2]

array([[    0, 17249],
       [    1, 18015],
       [    2, 14196],
       ...,
       [14135, 19938],
       [14136, 20214],
       [14137, 15542]], dtype=int32)

In [33]:
def build_test_set(itemsnoninteracted:list, gt_test_interactions: np.ndarray) -> list:
    #max_users, max_items = dims # number users (943), number items (2625)
    test_set = []
    for pair, negatives in tqdm(zip(gt_test_interactions, itemsnoninteracted), desc="BUILDING TEST SET..."):
        # APPEND TEST SETS FOR SINGLE USER
        negatives = np.delete(negatives, np.where(negatives == pair[1]))
        single_user_test_set = np.vstack([pair, ] * (len(negatives)+1))
        single_user_test_set[:, 1][1:] = negatives
        test_set.append(single_user_test_set.copy())
    return test_set

print(test_x[0])
test_x = build_test_set(items2compute, test_x[:,:2])
test_x[0]

[    0 17249     5]


BUILDING TEST SET...: 14138it [00:02, 5760.71it/s]


array([[    0, 17249],
       [    0, 14149],
       [    0, 14156],
       [    0, 14158],
       [    0, 14177],
       [    0, 14208],
       [    0, 14221],
       [    0, 14271],
       [    0, 14280],
       [    0, 14346],
       [    0, 14402],
       [    0, 14439],
       [    0, 14463],
       [    0, 14471],
       [    0, 14489],
       [    0, 14502],
       [    0, 14550],
       [    0, 14624],
       [    0, 14647],
       [    0, 14695],
       [    0, 14722],
       [    0, 14756],
       [    0, 14867],
       [    0, 14886],
       [    0, 14905],
       [    0, 14979],
       [    0, 14996],
       [    0, 15012],
       [    0, 15023],
       [    0, 15077],
       [    0, 15142],
       [    0, 15146],
       [    0, 15287],
       [    0, 15317],
       [    0, 15452],
       [    0, 15466],
       [    0, 15505],
       [    0, 15544],
       [    0, 15549],
       [    0, 15566],
       [    0, 15577],
       [    0, 15691],
       [    0, 15849],
       [   

# Building Factorization Machines model

In [34]:
class FM_operation(torch.nn.Module):

    def __init__(self, 
                 reduce_sum: bool=True) -> None:
        super().__init__()
        self.reduce_sum = reduce_sum

    def forward(self,
                x: torch.Tensor) -> float:
        """
        :param x: Float tensor of size ``(batch_size, num_fields, embed_dim)``
        """
        # square_of_sum = np.sum(x, dim=1) ** 2 # ...
        # sum_of_square = np.sum(x ** 2, dim=1) # ...
        
        square_of_sum = torch.pow(torch.sum(x, dim=1),2)
        sum_of_square = torch.sum(torch.pow(x,2), dim=1)
        ix = square_of_sum - sum_of_square
        if self.reduce_sum:
            ix = torch.sum(ix, dim=1, keepdim=True)
        return 0.5 * ix
        

In [35]:
class FactorizationMachineModel(torch.nn.Module):
    """
    A pytorch implementation of Factorization Machine.

    Reference:
        S Rendle, Factorization Machines, 2010.
    """

    def __init__(self, 
                 field_dims: list,
                 embed_dim: float) -> None:
        super().__init__()
        self.linear = torch.nn.Linear(len(field_dims), 1)
        self.embedding = torch.nn.Embedding(field_dims[-1], embed_dim)
        self.fm = FM_operation(reduce_sum=True)

        torch.nn.init.xavier_uniform_(self.embedding.weight.data)

    def forward(self, interaction_pairs: torch.Tensor) -> torch.Tensor:
        """
        :param interaction_pairs: Long tensor of size ``(batch_size, num_fields)``
        """
        out = self.linear(interaction_pairs.float()) + self.fm(self.embedding(interaction_pairs))
        return out.squeeze(1)
        
    def predict(self, 
                interactions: np.ndarray,
                device: torch.device) -> torch.Tensor:
        # return the score, inputs are numpy arrays, outputs are tensors
        test_interactions = torch.from_numpy(interactions).to(dtype=torch.long, device=device) #, dtype=torch.long)
        output_scores = self.forward(test_interactions)
        return output_scores

# Pipeline functions

## Training

In [36]:
from statistics import mean

def train_one_epoch(model: torch.nn.Module,
                    optimizer: torch.optim,
                    data_loader: torch.utils.data.DataLoader,
                    criterion: torch.nn.functional,
                    device: torch.device) -> float:
    model.train()
    total_loss = []

    for i, (interactions, targets) in enumerate(data_loader):
      
      interactions = interactions.to(device)
      targets = targets.to(device)

      predictions = model(interactions[:,:2])
  
      loss = criterion(predictions, targets.float())
      model.zero_grad()
      loss.backward()
      optimizer.step()
      total_loss.append(loss.item())

    return mean(total_loss)

# Define metrics

In [37]:
import math

def getHitRatio(recommend_list: list,
                gt_item: int) -> bool:
    if gt_item in recommend_list:
        return 1
    else:
        return 0

def getNDCG(recommend_list: list,
            gt_item: int) -> float:
    idx = np.where(recommend_list == gt_item)[0]
    if len(idx) > 0:
        return math.log(2)/math.log(idx+2)
    else:
        return 0

# Inference


In [44]:
def test(model: torch.nn.Module,
         test_x: np.ndarray,
         device: torch.device,
         topk: int=10) -> Tuple[float, float]:
    # Test the HR and NDCG for the model @topK
    model.eval()

    rcmnd_lst = np.zeros(len(test_x)).tolist()
    index = 0

    HR, NDCG = [], []
    for user_test in test_x:
        gt_item = user_test[0][1]
        predictions = model.predict(user_test, device)
        _, indices = torch.topk(predictions, topk)
        recommend_list = user_test[indices.cpu().detach().numpy()][:, 1]
        rcmnd_lst[index] = recommend_list.tolist().copy()
        index +=1

        HR.append(getHitRatio(recommend_list, gt_item))
        NDCG.append(getNDCG(recommend_list, gt_item))
    return mean(HR), mean(NDCG), rcmnd_lst

# PIPELINE
## Defining the model, the loss and the optimizer



In [39]:
dims = train_dataset.dims
model = FactorizationMachineModel(dims, hparams['hidden_size']).to(device)

criterion = torch.nn.BCEWithLogitsLoss(reduction='mean')
optimizer = torch.optim.Adam(params=model.parameters(), lr=hparams['learning_rate'])

## Random evaluation

In [40]:
import random
class RandomModel(torch.nn.Module):
    def __init__(self, 
                 dims: list) -> None:
        super(RandomModel, self).__init__()
        """
        Simple random based recommender system
        """
        self.all_items = list(range(dims[0], dims[1]))

    def forward(self) -> None:
        pass

    def predict(self,
                interactions: np.ndarray,
                device=None) -> torch.Tensor:
        return torch.FloatTensor(random.sample(self.all_items, len(interactions)))

rnd_model = RandomModel(dims)

In [46]:
# Popularity-Based Recommender System
class PopularityBasedModel(torch.nn.Module):
  """
  https://github.com/LaxmiChaudhary/Amzon-Product-Recommendation/blob/master/Recommendation%20System.ipynb
  The Popularity-based recommender system is a non-personalised recommender system and these are based on frequecy counts, which may be not suitable to the user.
  We can see the differance above for the user id 4, 6 & 8, The Popularity based model has recommended the same set of 5 products to both.
  """
  def __init__(self, 
                 popularity_recommendations) -> None:
        super(PopularityBasedModel, self).__init__()
        """
        Simple random based recommender system
        """
        self.popularity_recommendations = popularity_recommendations
  
  def predict(self,
              interactions: np.ndarray,
              device=None) -> torch.Tensor:
      return torch.FloatTensor(self.popularity_recommendations[:len(interactions)])


# train_data_sort = train_data_grouped.sort_values(['score', 'productId'], ascending = [0,1]) 
      
# #Generate a recommendation rank based upon score 
# train_data_sort['rank'] = train_data_sort['score'].rank(ascending=0, method='first') 
          
# #Get the top 5 recommendations 
# popularity_recommendations = train_data_sort.head(5) 


pop_model = PopularityBasedModel(popularity_recommendations)

## Final pipeline

In [42]:
data_loader = DataLoader(train_dataset, batch_size=hparams['batch_size'], shuffle=True, num_workers=0)

# Start training the model

In [47]:
# DO EPOCHS NOW
from datetime import datetime
save_data_configuration(datetime.now().strftime("%d-%b-%Y  %H:%M"))
time_start = time.time()
topk = 10
for epoch_i in range(hparams['num_epochs']):
    #data_loader.dataset.negative_sampling()
    train_loss = train_one_epoch(model, optimizer, data_loader, criterion, device)
    hr, ndcg, recommend_list_fm = test(model, test_x, device, topk=topk)
    
    print(save_data_configuration(f'MODEL: FACTORIZATION MACHINE'))
    print(save_data_configuration(f'epoch {epoch_i}:'))
    print(save_data_configuration(f'training loss = {train_loss:.4f} | Eval: HR@{topk} = {hr:.4f}, NDCG@{topk} = {ndcg:.4f} '))
 
    tb_fm.add_scalar('train/loss', train_loss, epoch_i)
    tb_fm.add_scalar(f'eval/HR@{topk}', hr, epoch_i)
    tb_fm.add_scalar(f'eval/NDCG@{topk}', ndcg, epoch_i)

    hr, ndcg, recommend_list_rnd = test(rnd_model, test_x, device, topk=topk)

    print(save_data_configuration(f'MODEL: RANDOM'))
    print(save_data_configuration(f'epoch {epoch_i}:'))
    print(save_data_configuration(f'training loss = {train_loss:.4f} | Eval: HR@{topk} = {hr:.4f}, NDCG@{topk} = {ndcg:.4f} '))
    save_data_configuration("_"*65)
 
    tb_rnd.add_scalar(f'eval/HR@{topk}', hr, epoch_i)
    tb_rnd.add_scalar(f'eval/NDCG@{topk}', ndcg, epoch_i)

    hr, ndcg, recommend_list_pop = test(pop_model, test_x, device, topk=topk)

    print(save_data_configuration(f'MODEL: POPULARITY-BASED'))
    print(save_data_configuration(f'epoch {epoch_i}:'))
    print(save_data_configuration(f'training loss = {train_loss:.4f} | Eval: HR@{topk} = {hr:.4f}, NDCG@{topk} = {ndcg:.4f} '))
    save_data_configuration("_"*65)
 
save_data_configuration(f"# Training duration: {(time.time()-time_start):.4f}")

MODEL: FACTORIZATION MACHINE
epoch 0:
training loss = 0.4724 | Eval: HR@10 = 0.2647, NDCG@10 = 0.1530 
MODEL: RANDOM
epoch 0:
training loss = 0.4724 | Eval: HR@10 = 0.0736, NDCG@10 = 0.0326 
MODEL: POPULARITY-BASED
epoch 0:
training loss = 0.4724 | Eval: HR@10 = 0.0105, NDCG@10 = 0.0031 
MODEL: FACTORIZATION MACHINE
epoch 1:
training loss = 0.4502 | Eval: HR@10 = 0.2739, NDCG@10 = 0.1586 
MODEL: RANDOM
epoch 1:
training loss = 0.4502 | Eval: HR@10 = 0.0777, NDCG@10 = 0.0355 
MODEL: POPULARITY-BASED
epoch 1:
training loss = 0.4502 | Eval: HR@10 = 0.0105, NDCG@10 = 0.0031 
MODEL: FACTORIZATION MACHINE
epoch 2:
training loss = 0.4290 | Eval: HR@10 = 0.2582, NDCG@10 = 0.1493 
MODEL: RANDOM
epoch 2:
training loss = 0.4290 | Eval: HR@10 = 0.0768, NDCG@10 = 0.0339 
MODEL: POPULARITY-BASED
epoch 2:
training loss = 0.4290 | Eval: HR@10 = 0.0105, NDCG@10 = 0.0031 
MODEL: FACTORIZATION MACHINE
epoch 3:
training loss = 0.4136 | Eval: HR@10 = 0.2415, NDCG@10 = 0.1401 
MODEL: RANDOM
epoch 3:
trainin

'# Training duration: 938.4538'

In [48]:
recommend_list_pop[0]

[17138, 14550, 18802, 17178, 15466, 19696, 18428, 17644, 18806, 18844]

In [49]:
recommend_list_pop[1]

[18336, 15379, 18435, 16388, 20012, 19174, 16781, 16192, 19621, 18015]

In [ ]:
# DO EPOCHS NOW
from datetime import datetime
save_data_configuration(datetime.now().strftime("%d-%b-%Y  %H:%M"))
time_start = time.time()
topk = 10
for epoch_i in range(hparams['num_epochs']):
    #data_loader.dataset.negative_sampling()
    train_loss = train_one_epoch(model, optimizer, data_loader, criterion, device)
    hr, ndcg, recommend_list_fm = test(model, test_x, device, topk=topk)
    
    print(save_data_configuration(f'MODEL: FACTORIZATION MACHINE'))
    print(save_data_configuration(f'epoch {epoch_i}:'))
    print(save_data_configuration(f'training loss = {train_loss:.4f} | Eval: HR@{topk} = {hr:.4f}, NDCG@{topk} = {ndcg:.4f} '))
 
    tb_fm.add_scalar('train/loss', train_loss, epoch_i)
    tb_fm.add_scalar(f'eval/HR@{topk}', hr, epoch_i)
    tb_fm.add_scalar(f'eval/NDCG@{topk}', ndcg, epoch_i)

    hr, ndcg, recommend_list_rnd = test(rnd_model, test_x, device, topk=topk)

    print(save_data_configuration(f'MODEL: RANDOM'))
    print(save_data_configuration(f'epoch {epoch_i}:'))
    print(save_data_configuration(f'training loss = {train_loss:.4f} | Eval: HR@{topk} = {hr:.4f}, NDCG@{topk} = {ndcg:.4f} '))
    save_data_configuration("_"*65)
 

    tb_rnd.add_scalar(f'eval/HR@{topk}', hr, epoch_i)
    tb_rnd.add_scalar(f'eval/NDCG@{topk}', ndcg, epoch_i)
save_data_configuration(f"# Training duration: {(time.time()-time_start):.4f}")

# Visualization

In [ ]:
recommend_list_fm[0]

In [50]:
def coverage(recommend_list, train_x):
  coverage = np.zeros(len(recommend_list)).tolist()
  items_train = np.unique(train_x[:,1])
  index = 0

  for items_per_user in recommend_list:
    recomend_items_unique = np.unique(items_per_user)
    coverage[index] = len(recomend_items_unique) / len(items_train) *100
    index+=1

  return coverage, sum(coverage)/len(coverage)

# coverage, mean_coverage = coverage(recommend_list_fm, train_x)
coverage, mean_coverage = coverage(recommend_list_pop, train_x)

In [51]:
mean_coverage

0.1612235422000889

In [ ]:

tb = tensorboard.program.TensorBoard()
tb.configure(bind_all=True, logdir=f"4_Modelling/{logs_base_dir}")
url = tb.launch()
webbrowser.open_new_tab(url.replace("MSI","localhost"))

In [ ]:
%tensorboard --logdir {logs_base_dir}